# ニューラルネットワークの数学 1（順伝播）

本講座（応用編）では、基礎編で学んだ機械学習の基礎から更に、ディープラーニングに特化して学んでいきます。  
まずはディープラーニングの基礎となるニューラルネットワークの数学から TensorFlow と呼ばれるオープンソースフレームワークを用いてのニューラルネットワークの実装方法について学びます。講座の後半では、ディープラーニングで用いられる画像・時系列・テキストデータの取り扱い方法、それぞれのデータを用いたディープラーニングの実装方法について学びます。   
ゴールとしては、ディープラーニングを研究やビジネスに活用できる基礎的なレベルを目指します。  

第 1 章では、ディープラーニングを理解する上で必要なニューラルネットワークの数学について学びます。 ニューラルネットワークの基礎となる計算の流れである順伝播の計算の中身を理解し、どのようにニューラルネットワークが予測値を算出するのかを理解します。  

## 本章の構成

- ニューラルネットワークの基礎
- 順伝播の計算

## ニューラルネットワークの基礎

ディープラーニング (deep Learning) とは多層のニューラルネットワーク (neural network) による機械学習の手法を指します。そのため、ディープラーニングを理解する上でニューラルネットワークについて理解することは重要です。（ディープラーニングの定義の詳細に関しては[こちら](https://ja.wikipedia.org/wiki/%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0)を参照して下さい。）  
まずはこのニューラルネットワークの概要や構造、そしてその種類について全体像を理解します。そして、次節の順伝播の計算で詳細の計算方法などを確認します。 

### ニューラルネットワークの概要
ニューラルネットワークは回帰や分類といった教師あり学習のアルゴリズムのひとつです。そのため、入力変数から出力変数を予測する構造は同じです。今回は問題設定として、入力変数に家に関する情報（部屋の広さ、駅からの距離、築年数）を取り、出力変数は家賃とします。物件情報から家賃を予測する回帰の問題設定にします。

最初は計算の詳細はおいておき、左から右にむけて何らかの計算を内部で行うことにより、予測値を出力するというイメージを掴みましょう。

![ニューラルネットワークの構造 1](http://drive.google.com/uc?export=view&id=1NmUZcB4rG7i7n46oc9d9Zepl-89GUUOh)

左に入力変数が入っており、右に予測したい家賃の値が出力されます。この丸で表されたものを**ノード (node)** と呼びます。そしてノードの縦方向の集まりのことを**層 (layer)** と呼びます。左側から**入力層 (input layer)**、**中間層 (intermediate layer)**、**出力層 (output layer)** と呼び、特に中間層は**隠れ層 (hidden layer)** といいます。プログラミングをする際に英語での表記を使用しますので、英語でも覚えておきましょう。

上のニューラルネットワークは、入力層、中間層、出力層がそれぞれ一つずつあるので、合計 3 層の構造となっています。中間層をさらに多層にしたもの、2 層以上のものがディープラーニングと呼ばれています。

### パラメータ

![ニューラルネットワークの構造 2](http://drive.google.com/uc?export=view&id=1iW2N4VXZpRjvL0cm2SoZLq1j7YjfQ7zw)

ここでは入力層と中間層に着目しましょう。直線がそれぞれのノードを接続していますが、これをエッジと呼び、重み (weight) を表します。重みは新しく登場した概念ではなく、重回帰分析などのアルゴリズムでも登場してきたもの同様に**学習によって最適化したい値**です。  
バイアスは図のように層の最後に追加されたノードのエッジによって表現されます。入力変数以外の値で補正する意味合いを持っています。

### ニューラルネットワークの種類
ニューラルネットワークには、上図のような層間のノードが互いに密に結合した**全結合型 (fully-connected)** のものだけでなく、画像処理などでよく用いられる**畳み込み型 (convolutional)**、自然言語や系列データの扱いによく用いられる**再帰型 (recurrent)** など、いくつもの種類が存在します。それらの違いは基本的には層間の結合の仕方にあり、別々の層間には別の種類の結合の仕方を用いることができます。つまり、単一のニューラルネットワークの中に複数の種類の結合が混ざって現れることもよくあります。

例えば、画像処理でよく使われる畳み込みニューラルネットワーク (Convolutional Neural Network) では入力に近い層では畳み込み型の結合を用い、出力に近い層では全結合型の結合を用います。

本章では、最も基本となる全結合型のみ着目して学んでいきます。その他の種類については、画像認識の基礎編など後の章で学んでいきます。

## 順伝播の計算
順伝播とは、ニューラルネットワークにおいて入力層から出力層への方向を順方向として入力変数とパラメータをかけ合わせて予測値を計算することを指します。  
そして、その具体的な計算では主に 2 つの計算、**線形変換**と**非線形変換**を繰り返します。これらをそれぞれ紹介していきます。

### 線形変換
それではまず線形変換から紹介します。線形変換ではこれまでに紹介していた考え方と同じで、重みと入力変数の線形結合を行います。下記の図のように、入力層のノードが持つ値（入力値）と重みを掛け合わせ、それぞれのノードの計算結果を足し合わせます。今回は簡略化のためにバイアスの計算は省略して考えます。

![ニューラルネットワークの計算 1](http://drive.google.com/uc?export=view&id=1D5wTWRLvtTHxaeQKKFsAGDBFK0Wpy9HT)

線形変換は上記の図のように重みと入力値の線形結合になっています。次にこれらを数式に当てはめて確認してみましょう。

下記の図は値を文字に置き換えたものです。

![ニューラルネットワークの計算 2](http://drive.google.com/uc?export=view&id=18UG8BFn0Drd25JmSKyQFy4n2z2Tjv0Jh)

上記の図で線形変換後の変数 $u_{11}$ と $u_{22}$ は以下のように計算されます。

$$
\begin{aligned}
u_{11} = w_{11}x_{1} + w_{12}x_{2} + w_{13}x_{3} + b_{1} \\
u_{12} = w_{21}x_{1} + w_{12}x_{2} + w_{13}x_{3} + b_{2}
\end{aligned}
$$

バイアスに対応する $x_0$ は 1 が入るため省略しています。そしてこの式は、入力ベクトル $\mathbf{x}$、重み行列 $\mathbf{W}$、バイアスベクトル $\mathbf{b}$、および出力ベクトル  $\mathbf{u}$ を

$$
\begin{aligned}
\mathbf{x} &= \begin{bmatrix}x_{1} \\ x_{2} \\ x_{3} \end{bmatrix} \\\\
\mathbf{W} &= \begin{bmatrix}w_{11} &w_{12}&w_{13}\\ w_{21}&w_{22}&w_{23}\end{bmatrix} \\\\
\mathbf{b} &= \begin{bmatrix}b_{1} \\ b_{2}\end{bmatrix} \\\\
\mathbf{u} &= \begin{bmatrix}u_{11} \\ u_{12}\end{bmatrix} 
\end{aligned}
$$

と定義すれば、

$$
\begin{aligned}
\begin{bmatrix}u_{11} \\ u_{12}\end{bmatrix} &= \begin{bmatrix}w_{11} &w_{12}&w_{13}\\ w_{21}&w_{22}&w_{23}\end{bmatrix} \begin{bmatrix}x_{1} \\ x_{2} \\ x_{3}\end{bmatrix} + \begin{bmatrix}b_{1} \\ b_{2}\end{bmatrix} \\\\
\mathbf{u} &= \mathbf{W} \, \mathbf{x} + \mathbf{b}
\end{aligned}
$$

のように表現することができます。これが線形変換の計算になります。行列とベクトルを用いた形式でまとめることができました。重回帰分析では $\mathbf{w}^{\mathrm T} \mathbf{x}+b$ であったところから、さらに拡張された表現であることがわかります。続いてはこの線形変換によって出力された値を用いて非線形変換を行います。非線形変換がどのような計算をしていて、なぜ必要なのか学びます。

### 非線形変換
下記の図をご覧ください。図の左に示すように線形変換では入出力間が線形な関係性を持つ場合のみその関係性を表現することができます。その一方で、下図の右に示すような入出力間が非線形な関係を保つ場合には、その関係性をうまく捉えることができません。

![線形と線形非線形 1](http://drive.google.com/uc?export=view&id=15q_FHrR6vGFffHTjD13sZxixEwcr15Zg)

例えば、線形変換のみで図の右の白い丸で表される観測データから $x$ と $y$ の関係を近似しようとした場合、点線のような直線が得られるとします。これでは、一部のデータしか当てはまっていないことが分かります。

そこでニューラルネットワークでは各層において、線形変換に続いて非線形変換を施し、層を積み重ねて作られるニューラルネットワーク全体としても非線形性を持つことができるようにしています。この非線形変換を行う関数のことを、ニューラルネットワークの文脈では**活性化関数**と呼びます。

線形変換の結果 $u_{11}$, $u_{12}$ に活性化関数を使って非線形変換を行った結果を $z_{11}$, $z_{12}$ と書き、これらを**活性値 （activation）** と呼びます（下図参照）。 これが次の層への入力となります。

![線形と線形非線形 2](http://drive.google.com/uc?export=view&id=1K2pVqTLfECfiPhxtTIJX-LkQDB4DicxJ)

最もよく知られている活性化関数のひとつは、下図に示す**ロジスティックシグモイド関数 (logistic sigmoid function)**（以下、シグモイド関数）です。この活性化関数が従来、ニューラルネットワークでよく用いられてきました。入力変数 $u$ に基づいて、0~1 の間に値を変換します。関数の形から分かる通り、直線では表現できないような構造であるため、入出力間の非線形性を一部では捉えることができます。

![シグモイド関数](http://drive.google.com/uc?export=view&id=1GRjTmnpI3S__ZXmuIXRHDchRivKlgWJf)

数式では、

$$
\begin{array}{c} z(u) = \frac{1}{1+\exp(-u)} \end{array}
$$

と表されます。

しかし近年、層の数が多いニューラルネットワークではシグモイド関数は活性化関数としてほとんど用いられません。
その理由の一つは、シグモイド関数を活性化関数に採用すると**勾配消失（vanishing gradient）**という現象が起きやすくなり、学習が進行しなくなる問題が発生することがあるためです。ここで勾配消失の詳細の説明は行いませんので、詳しくは[こちら](https://ja.wikipedia.org/wiki/%E3%83%87%E3%82%A3%E3%83%BC%E3%83%97%E3%83%A9%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0#%E5%8B%BE%E9%85%8D%E6%B6%88%E5%A4%B1%E5%95%8F%E9%A1%8C)を確認して下さい。  
この問題を回避するために、**正規化線形関数（rectified linear unit, ReLU）**がよく用いられています。
これは、以下のような形をした関数です。

![ReLU関数](http://drive.google.com/uc?export=view&id=1wWHNVQO2eLSPOIXg67FGglyL1JK-TQSg)

$$
\begin{array}{c} z(u) = \max(0, u) \end{array}
$$

ここで、$\max(0, u)$ は、$0$ と $u$ を比べて**大きな方の値を返す**関数です。すなわち、**ReLU は入力が負の値の場合には出力は 0 で一定**であり、**正の値の場合は入力をそのまま出力する**という関数です。  シグモイド関数では、入力が $0$ から離れた値をとると、どんどん曲線の傾き（勾配）が小さくなっていくことが一つ前の図からも見て取れます。それに対し、ReLU 関数は入力の値が正であれば、いくら大きくなっていっても、傾き（勾配）は一定です。これがシグモイド関数で問題となっていた勾配消失に対して有効に働きます。

非線形変換の最後に、**ソフトマックス関数 (softmax)** を紹介します。主に分類の問題で使用される活性化関数であり、出力層の活性化関数として用いられます。数式は、

$$
\begin{array}{c} a(u) = \frac{\exp(u)}{\sum_{m=1}^{M} \exp(u_m)} \end{array}
$$

のように表され、$u_m$ は複数からなる入力値です。ここで、$M$ は変換後の層のノード数であり、$\exp(\cdot)$ は自然対数 $e$ を底とした指数関数を表しています。

ソフトマックス関数の数式自体は難しく見えるかもしれませんが、その機能は**出力される値の総和を 1 に正規化する**ことです。  
分類の場合、ニューラルネットワークから出力される変数の総和が 1 となり、それぞれの出力値は 0~1 の間で値を取ります。

分類の場合にソフトマックス関数は頻出するため覚えておきましょう。

### 数値例で確認
一度具体的な数値例で計算の流れを確認しましょう。下記の図を参照して、予測値 $y$ の値を求めて下さい。なお、$\mathbf{u}$ から $\mathbf{z}$ への非線形変換は ReLU 関数を用いる事とします。

![順伝播の計算](http://drive.google.com/uc?export=view&id=1DBajl5Iu9AhbClfa8-9vxwx8F2kbc1Qp)


ここでニューラルネットワークと重回帰分析の間に大きな違いがあります。ニューラルネットワークも重回帰分析も目的関数を最適化するようにパラメータである $\mathbf{w}$ や $b$ を決定することが目的です。重回帰分析はこれを達成するために目的関数を微分して勾配を求め、その勾配が $0$ ベクトルとなるようにパラメータを決定していました。ニューラルネットワークも同じように進めれば良いように関しますが、大きな違いがあります。その謎は活性化関数に隠されています。ReLU 関数には $\max(\cdot, \cdot)$ の関数を持っており、この関数はどのように微分すれば良いでしょうか。入力される値によって結果が異なるため、値をいれない代数の段階ではこの微分を行うことができません。したがって、重回帰分析のようにパラメータを直接求めることができません。

ここが一番のポイントなのですが、ニューラルネットワークを含めたいくつかのアルゴリズムでは、**パラメータの初期値をランダムに決定**し、そこから**徐々に調整していく**という方法を取ります。現時点では、パラメータの初期値がないと計算が始まらないと認識してください。そのため、今回の数値例においても、パラメータの初期値をランダムに設定しています。バイアス $b$ の初期値は 0 と設定されることが多いため、今回は 0 としています。

今回は入力として $\mathbf{x}=[1, 2, 3]^{\mathrm T}$ が与えられているため、$\mathbf{u}$ を求める計算式は下記になります。

$$
\begin{aligned} u_{11} &= 3 \times 1 + 1 \times 2 + 2 \times 3 + 0 = 11 \\ u_{12} &= (-2) \times 1 + (-3) \times 2 + (-1) \times 3 + 0 = - 11 \end{aligned}
$$

と求まります。次に $z_{11}$ の値を ReLU 関数を適用し、

$$
\begin{aligned} z_{11} &= \max(0, u_{11}) = \max(0, 11) = 11 \\ z_{12} &= \max(0, u_{12}) = \max(0, -11) = 0 \end{aligned}
$$

と求まります。これらの値をもとに予測値 $y$ は最初の線形変換と同じ計算方法で、

$$
\begin{aligned} y = 2\times11 + 3\times0 + 0 = 22 \end{aligned}
$$

と計算できました。ニューラルネットワークの実際の計算は足し算と掛け算ぐらいで構成されているため、計算自体は非常に簡単であることがわかります。

### 目的関数
実際に数値例で確認をし、予測値 $y$ は $22$ と算出されました。もしも、このときの答え、目的変数 $t$ の値が $10$ だった場合、パラメータはどのように更新されると望ましい結果になるでしょうか。

![平均二乗誤差](http://drive.google.com/uc?export=view&id=19heS7k5t7gP-ejtjSnyUiJBpnhkBqsMZ)

目標値 $t$ と予測値 $y$ の誤差が最も小さくなった場合が最も望ましい結果であるといえます。ニューラルネットワークでは、この誤差を問題設定に応じて計算し、パラメータの調整を行っていきます。このニューラルネットワークに用いられる誤差を求める関数を**目的関数**と呼びます。損失関数、コスト関数と呼ばれることも多いですが、それぞれ少し概念は異なり、目的関数が最も大きな概念です。今回は目的関数と統一して紹介します。

ニューラルネットワークの学習には、**微分可能**であれば、解きたいタスクに合わせて様々な目的関数を利用することができます。ここでは、

- 回帰でよく用いられる**平均ニ乗誤差 (mean squared error)**
- 二クラス分類でよく用いられる**二値交差エントロピー (binary cross entropy)**
- 多クラス分類でよく用いられる**交差エントロピー (cross entropy)**

という代表的な 3 つの目的関数を紹介します。

### 平均二乗誤差
**平均ニ乗誤差 (mean squared error)** は、回帰によく用いられる目的関数です。重回帰分析の二乗和誤差と似ていますが、各データ点における誤差の総和をとるだけでなく、それをデータ数で割って、誤差の平均値を計算している点が異なります。式で表すと、以下のようになります。

$$
\begin{array}{c} \mathcal L = \frac{1}{N} \sum_{n=1}^{N}(t_n - y_n)^2 \end{array}
$$

$N$ はサンプル数、$y_n$ は $n$ 個目のデータに対するニューラルネットワークの出力値（予測値）、$t_n$ は $n$ 個目のデータに対する望ましい正解の値です。今回の数値例で平均ニ乗誤差を算出すると、

$$
\begin{array}{c} \mathcal L = \frac{1}{1}\,(10 - 22)^2 = 144 \end{array}
$$

と計算できます。今回は予測に用いるサンプル数が 1 つであるため $N$ は 1 となっています。

### 二値交差エントロピー

犬と猫の二値分類のような場合には二値交差エントロピーを用います。二値交差エントロピーは、

$$
\begin{array}{c} \mathcal L = - \sum_{n=1}^{N} t_n \log y_n + (1-t_n)\log(1-y_n) \end{array}
$$

で表されます。目的変数 $t_n$ は $0$ か $1$ が与えられ、予測値 $y_n$ は二値分類の場合は多くの場合にシグモイド関数が施されるため $0$ と $1$ の間で値を取ります。

$t_n \log y_n + (1-t_n)\log(1-y_n)t$ に注目して、もう少し深堀りを行いましょう。二値交差エントロピーの最小化問題を考えますが、式全体に $-$ がついているため、この項目は最大化を行いたいことがわかります。$t_n=0$ のときには、

$$
\begin{array}{c} \log(1 - y_n) \end{array}
$$

となり、$y_n$ が $0$ に近いほど値が大きくなります。$t_n=1$ のときには、

$$
\begin{array}{c} \log y_n \end{array}
$$

となり、$y_n$ が $1$ に近いほど値が大きくなります。


この確率の考え方や $\log$ が登場する背景などは確率や情報量を勉強すると理解が深まりますが、現状では二値分類では目的関数として二値交差エントロピーを使用しているという程度で問題ありません。確率や情報量の数学はこれまでの数学よりも多少難易度が高いため、一通り学び終えたころで理解を進めてください。

### 交差エントロピー

**交差エントロピー (cross entropy)** は、多クラス分類を解きたい際によく用いられる目的関数であり、二値交差エントロピーを拡張したものです。例として、$K$ クラス分類を考えてみましょう。ある入力 $\mathbf{x}$ が与えられたとき、ニューラルネットワークの出力層に $K$ 個のノードがあり、それぞれがこの入力が $k$ 番目のクラスに属する確率

$$
\begin{array}{c} y_k = p(y = k|\mathbf{x}) \end{array}
$$

を表しているとします。これは、入力 $\mathbf{x}$ が与えられたという条件のもとで、予測クラスを意味する $y$ が $k$ であるような確率を表す**条件付き確率**です。

ここで、$\mathbf{x}$ が所属するクラスの正解が、

$$
\begin{array}{c} \mathbf{t} = \begin{bmatrix} t_1 & t_2 & \cdots & t_k \end{bmatrix}^{\mathrm T} \end{array}
$$

というベクトルで与えられているとします。ただし、このベクトルは $t_k (k=1, 2, \cdots, K)$ の**いずれか 1 つだけが 1 であり、それ以外は 0 であるようなベクトル**であるとします。これを**ワンホットベクトル (one-hot vector)** と呼びます。そして、この 1 つだけが値が 1 となっている要素は、その要素のインデックスに対応したクラスが正解であることを意味します。例えば、$t_3 =1$ であれば 3 つ目のクラス（3 というインデックスに対応するクラス）が正解であるということになります。

以上を用いて、交差エントロピーは以下のように定義されます。

$$
\begin{array}{c} - \sum_{k=1}^{K}t_k \log y_k \end{array}
$$

これは $t_k$ が $k=1, \cdots, K$ のうち正解クラスである一つの $k$ の値でだけ $1$ となるので、正解クラスであるような $k$ での $\log y_k$ を取り出して $-1$ を掛けているのと同じです。また、$N$ 個すべてのサンプルを考慮すると、交差エントロピーは、

$$
\begin{array}{c} \mathcal L = - \sum_{n=1}^{N} \sum_{k=1}^{K} t_{n, k} \log y_{n,k} \end{array}
$$

となります。

## 練習問題 本章のまとめ

本章で学んだ内容を復習しましょう。問題の答えをそれぞれのセルに記載してください。  
（プログラミングを行うのではなく、問題をノート上などで解き、回答を空白のセルに記入してください。）

下記のニューラルネットワークの計算を行い平均二乗誤差を算出して下さい。また、計算途中の $\bf u_1$ 、 $\bf z_1$ 、$y$ の値も解答欄に記述して下さい。  
計算内容については下記の補足を確認して下さい。    

![ニューラルネットワークの計算 練習問題](http://drive.google.com/uc?export=view&id=1YgHrd6TYA6aNx2J_TJGU_PYYO7iZ6zec)

*補足*  

- バイアスは考慮しないで下さい。
- 活性化関数には ReLU 関数を使用して下さい。
- 目的関数は平均二乗誤差を使用して下さい。  

In [ ]:
# u11


In [ ]:
# u12


In [ ]:
# z11


In [ ]:
# z12


In [ ]:
# y


In [ ]:
# 平均二乗誤差


---
© 株式会社キカガク及び国立大学法人 豊橋技術科学大学